In [1]:
import pulp
import pandas as pd
import chardet
import os
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import time
import json

#### Optimiser

In [ ]:
def solve_multi_period_fpl(budget, data, start_gameweek, end_gameweek, season, bench_boost_gameweek = 0, free_hit_gameweek = 0):

    # Validate bench_boost_gameweek: 0 means no bench boost.
    if bench_boost_gameweek != 0 and (bench_boost_gameweek < start_gameweek -1 or bench_boost_gameweek > end_gameweek +1):
        raise ValueError("bench_boost_gameweek must be 0 or between start_gameweek and end_gameweek")
    # Validate free_hit_gameweek: 0 means no free hit; also, free hit should not be in the first gameweek.
    if free_hit_gameweek != 0:
        if free_hit_gameweek < start_gameweek or free_hit_gameweek > end_gameweek:
            raise ValueError("free_hit_gameweek must be 0 or between start_gameweek and end_gameweek")
        if free_hit_gameweek == start_gameweek:
            raise ValueError("free_hit_gameweek cannot be the first gameweek")
    
    # Filter data for the specified gameweeks and season.
    filtered_data = data[(data['gameweek'] >= start_gameweek) & 
                         (data['gameweek'] <= end_gameweek) & 
                         (data['season'] == season)]
    
    # Create a dictionary for quick access to player data.
    player_data = {(row['name'], row['gameweek']): row for _, row in filtered_data.iterrows()}
    
    # Create the optimization model.
    model = pulp.LpProblem("Multi_Period_FPL_Optimization", pulp.LpMaximize)

    # Sets.
    gameweeks = range(start_gameweek, end_gameweek + 1)
    players = filtered_data['name'].unique()
    positions = filtered_data['position'].unique()
    teams = filtered_data['team'].unique()

    # Decision Variables.
    squad = pulp.LpVariable.dicts("squad", [(p, gw) for p in players for gw in gameweeks], cat='Binary')
    lineup = pulp.LpVariable.dicts("lineup", [(p, gw) for p in players for gw in gameweeks], cat='Binary')
    captain = pulp.LpVariable.dicts("captain", [(p, gw) for p in players for gw in gameweeks], cat='Binary')
    vicecap = pulp.LpVariable.dicts("vicecap", [(p, gw) for p in players for gw in gameweeks], cat='Binary')
    transfer_in = pulp.LpVariable.dicts("transfer_in", [(p, gw) for p in players for gw in gameweeks], cat='Binary')
    transfer_out = pulp.LpVariable.dicts("transfer_out", [(p, gw) for p in players for gw in gameweeks], cat='Binary')
    free_transfers = pulp.LpVariable.dicts("free_transfers", gameweeks, lowBound=0, upBound=5, cat='Integer')
    paid_transfers = pulp.LpVariable.dicts("paid_transfers", gameweeks, lowBound=0, cat='Integer')

    # Objective:
    # - In the bench boost week (if > 0) use the entire squad (15 players), otherwise just the starting lineup.
    # - Transfer costs are applied in all weeks except the free hit week.
    model += pulp.lpSum(
        player_data.get((p, gw), {}).get('xP', 0) * (
            (squad[p, gw] if bench_boost_gameweek > 0 and gw == bench_boost_gameweek else lineup[p, gw])
            + captain[p, gw] + 0.1 * vicecap[p, gw]
        )
        for p in players for gw in gameweeks
    ) - pulp.lpSum(
        4 * paid_transfers[gw] for gw in gameweeks if gw != free_hit_gameweek
    )
    
    # Starting free transfers (typically 0 for the first gameweek).
    model += free_transfers[start_gameweek] == 0

    # Constraints that apply in every gameweek.
    for gw in gameweeks:
        # Squad size.
        model += pulp.lpSum(squad[p, gw] for p in players) == 15

        # Starting lineup size.
        model += pulp.lpSum(lineup[p, gw] for p in players) == 11

        # Exactly one captain and one vice-captain.
        model += pulp.lpSum(captain[p, gw] for p in players) == 1
        model += pulp.lpSum(vicecap[p, gw] for p in players) == 1

        # Lineup, captain, and vice-captain must be members of the squad.
        for p in players:
            model += lineup[p, gw] <= squad[p, gw]
            model += captain[p, gw] <= lineup[p, gw]
            model += vicecap[p, gw] <= lineup[p, gw]
            model += captain[p, gw] + vicecap[p, gw] <= 1

        # Position constraints.
        for pos in positions:
            # Ensure the correct number of players per position in the squad.
            model += pulp.lpSum(squad[p, gw] for p in players 
                                if player_data.get((p, gw), {}).get('position') == pos) \
                     == {'GK': 2, 'DEF': 5, 'MID': 5, 'FWD': 3}[pos]
            # For the starting lineup, enforce positional minimums/maximums.
            if pos == 'GK':
                model += pulp.lpSum(lineup[p, gw] for p in players 
                                    if player_data.get((p, gw), {}).get('position') == pos) == 1
            elif pos in ['DEF', 'MID']:
                model += pulp.lpSum(lineup[p, gw] for p in players 
                                    if player_data.get((p, gw), {}).get('position') == pos) >= 3
                model += pulp.lpSum(lineup[p, gw] for p in players 
                                    if player_data.get((p, gw), {}).get('position') == pos) <= 5
            else:  # FWD.
                model += pulp.lpSum(lineup[p, gw] for p in players 
                                    if player_data.get((p, gw), {}).get('position') == pos) >= 1
                model += pulp.lpSum(lineup[p, gw] for p in players 
                                    if player_data.get((p, gw), {}).get('position') == pos) <= 3

        # Budget constraint.
        model += pulp.lpSum(player_data.get((p, gw), {}).get('value', 0) * squad[p, gw] for p in players) <= budget

        # Team limit (max 3 players from the same team).
        for team in teams:
            model += pulp.lpSum(squad[p, gw] for p in players 
                                if player_data.get((p, gw), {}).get('team') == team) <= 3

        # Transfer constraints.
        # Handle free hit weeks specially.
        if gw > start_gameweek:
            # ----- Free Hit Week -----
            if free_hit_gameweek != 0 and gw == free_hit_gameweek:
                # In the free hit week, allow a completely new squad.
                # Do not enforce the continuity constraint.
                model += free_transfers[gw] == free_transfers[gw-1] 
                # (Transfers in/free hit week are not counted, so we do not add transfer_in/out constraints.)
            
            # ----- Week After Free Hit: Reversion -----
            elif free_hit_gameweek != 0 and gw == free_hit_gameweek + 1:
                # Force the squad to revert to the team from before the free hit.
                for p in players:
                    model += squad[p, gw] == squad[p, free_hit_gameweek - 1]
                # Also reset free transfers as if no transfers had been made in the free hit week.
                model += free_transfers[gw] == free_transfers[free_hit_gameweek - 1] #+ 1
            
            # ----- Normal Weeks -----
            else:
                model += free_transfers[gw] == free_transfers[gw-1] + 1 - pulp.lpSum(transfer_in[p, gw-1] for p in players)
                model += free_transfers[gw] <= 5 
                model += pulp.lpSum(transfer_in[p, gw] for p in players) == pulp.lpSum(transfer_out[p, gw] for p in players)
                model += pulp.lpSum(transfer_in[p, gw] for p in players) == free_transfers[gw] + paid_transfers[gw]
                for p in players:
                    model += squad[p, gw] == squad[p, gw-1] + transfer_in[p, gw] - transfer_out[p, gw]
                    model += transfer_in[p, gw] + transfer_out[p, gw] <= 1

    # Solve the model.
    model.solve()

    # Process and return results.
    if pulp.LpStatus[model.status] == 'Optimal':
        results = []
        for gw in gameweeks:
            picks = []
            transfers_in = []
            transfers_out = []
            for p in players:
                if (squad[p, gw].value() or 0) > 0.5:
                    player_info = player_data.get((p, gw), {})
                    is_captain = 1 if (captain[p, gw].value() or 0) > 0.5 else 0
                    is_lineup = 1 if (lineup[p, gw].value() or 0) > 0.5 else 0
                    is_vice = 1 if (vicecap[p, gw].value() or 0) > 0.5 else 0
                    picks.append([
                        p,
                        player_info.get('position', ''),
                        player_info.get('team', ''),
                        player_info.get('value', 0),
                        player_info.get('xP', 0),
                        is_lineup,
                        is_captain,
                        is_vice
                    ])
                if gw > start_gameweek:
                    if (transfer_in[p, gw].value() or 0) > 0.5:
                        transfers_in.append(p)
                    if (transfer_out[p, gw].value() or 0) > 0.5:
                        transfers_out.append(p)

            picks_df = pd.DataFrame(picks, columns=['name', 'pos', 'team', 'price', 'xP', 'lineup', 'captain', 'vicecaptain'])
            picks_df = picks_df.sort_values(by=['lineup', 'pos', 'xP'], ascending=[False, True, False])
            
            results.append({
                'gameweek': gw,
                'picks': picks_df,
                'transfers_in': transfers_in,
                'transfers_out': transfers_out,
                'free_transfers': free_transfers[gw].value(),
                'paid_transfers': paid_transfers[gw].value()
            })

        total_xp = pulp.value(model.objective)
        print(f'Total expected points for all gameweeks: {total_xp}')

        return {'results': results, 'total_xp': total_xp}
    else:
        print(f"Optimization failed. Status: {pulp.LpStatus[model.status]}")
        return None